# Youth Survey EDA 

We want to perform some preprocessing so we can use the data in Tableau. 

In [17]:
# Import necessary libaries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import sys 
# sys.path.append('..')
sys.path.append('C:\\Users\\jzqma\\Documents\\GithubRepos\\young-people-survey')   

In [18]:
# Import Datasets
from youth_survey.getsurvey import Survey

# Calling an instance
survey = Survey()

# Show all filenames in bucket
survey.get_all_filenames()

columns.csv
responses.csv


In [19]:
# converting each csv to dataframes 
columns = survey.get_csv('columns.csv')
responses = survey.get_csv('responses.csv')

## Check Out The Data 

In [21]:
print(columns.shape)
columns.head()

(150, 2)


,original,short
0,I enjoy listening to music.,Music
1,I prefer.,Slow songs or fast songs
2,"Dance, Disco, Funk",Dance
3,Folk music,Folk
4,Country,Country


In [24]:
print(columns.shape)
responses.head()

(150, 2)


,Music,Slow songs or fast songs,Dance,Folk,Country,Classical music,Musical,Pop,Rock,Metal or Hardrock,...,Age,Height,Weight,Number of siblings,Gender,Left - right handed,Education,Only child,Village - town,House - block of flats
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,20.0,163.0,48.0,1.0,female,right handed,college/bachelor degree,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,19.0,163.0,58.0,2.0,female,right handed,college/bachelor degree,no,city,block of flats
2,5.0,5.0,2.0,2.0,3.0,4.0,5.0,3.0,5.0,3.0,...,20.0,176.0,67.0,2.0,female,right handed,secondary school,no,city,block of flats
3,5.0,3.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,...,22.0,172.0,59.0,1.0,female,right handed,college/bachelor degree,yes,city,house/bungalow
4,5.0,3.0,4.0,3.0,2.0,4.0,3.0,5.0,3.0,1.0,...,20.0,170.0,59.0,1.0,female,right handed,secondary school,no,village,house/bungalow


Lets add back the [original questions](https://www.kaggle.com/miroslavsabo/young-people-survey) back to the `responses` dataframe. 

In [27]:
# Adding back the original question to the responses df 
responses.columns = columns['original']

# sanity check
responses.head(2)

original,I enjoy listening to music.,I prefer.,"Dance, Disco, Funk",Folk music,Country,Classical,Musicals,Pop,Rock,"Metal, Hard rock",...,Age,Height,Weight,How many siblings do you have?,Gender,I am,Highest education achieved,I am the only child,I spent most of my childhood in a,I lived most of my childhood in a
0,5.0,3.0,2.0,1.0,2.0,2.0,1.0,5.0,5.0,1.0,...,20.0,163.0,48.0,1.0,female,right handed,college/bachelor degree,no,village,block of flats
1,4.0,4.0,2.0,1.0,1.0,1.0,2.0,3.0,5.0,4.0,...,19.0,163.0,58.0,2.0,female,right handed,college/bachelor degree,no,city,block of flats


I want to separate the questionnaire into their respective groups. I will make a preference dictionary. 

The variables can be split into the following groups: 
* Music preferences (19 items)
* Movie preferences (12 items)
* Hobbies & interests (32 items)
* Phobias (10 items)
* Health habits (3 items)
* Personality traits, views on life, & opinions (57 items)
* Spending habits (7 items)
* Demographics (10 items)

In [26]:
# Groups can be from on the Kaggle Dataset Homepage.
preference_dict =  {'Music Preference': list(columns['original'][0:19]), 
                    'Movies Preference': list(columns['original'][19:31]),
                    'Hobbies': list(columns['original'][31:63]),
                    'Phobias': list(columns['original'][63:73]),
                    'Health Habits': list(columns['original'][73:76]), 
                    'Personality Traits': list(columns['original'][76:133]), 
                    'Spending Habits': list(columns['original'][133:140]), 
                    'Demographics': list(columns['original'][140:])} 

# sanity check
preference_dict

{'Music Preference': ['I enjoy listening to music.',
  'I prefer.',
  'Dance, Disco, Funk',
  'Folk music',
  'Country',
  'Classical',
  'Musicals',
  'Pop',
  'Rock',
  'Metal, Hard rock',
  'Punk',
  'Hip hop, Rap',
  'Reggae, Ska',
  'Swing, Jazz',
  'Rock n Roll',
  'Alternative music',
  'Latin',
  'Techno, Trance',
  'Opera'],
 'Movies Preference': ['I really enjoy watching movies.',
  'Horror movies',
  'Thriller movies',
  'Comedies',
  'Romantic movies',
  'Sci-fi movies',
  'War movies',
  'Tales',
  'Cartoons',
  'Documentaries',
  'Western movies',
  'Action movies'],
 'Hobbies': ['History',
  'Psychology',
  'Politics',
  'Mathematics',
  'Physics',
  'Internet',
  'PC Software, Hardware',
  'Economy, Management',
  'Biology',
  'Chemistry',
  'Poetry reading',
  'Geography',
  'Foreign languages',
  'Medicine',
  'Law',
  'Cars',
  'Art',
  'Religion',
  'Outdoor activities',
  'Dancing',
  'Playing musical instruments',
  'Poetry writing',
  'Sport and leisure activitie

## Data Wrangling
I will sepearte the numeric and categorical answers into two different dataframes. The categorical answers will later be used as new columns in the final dataframe. 

In [28]:
numeric_answers = responses.T.reset_index(drop=True)[0:140]
numeric_answers.head()

,0,1,2,3,4,5,6,7,8,9,...,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009
0,5.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,5.0,5.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,5.0
1,3.0,4.0,5.0,3.0,3.0,3.0,5.0,3.0,3.0,3.0,...,3.0,5.0,5.0,3.0,3.0,2.0,4.0,3.0,3.0,5.0
2,2.0,2.0,2.0,2.0,4.0,2.0,5.0,3.0,3.0,2.0,...,3.0,5.0,3.0,4.0,4.0,5.0,5.0,1.0,3.0,4.0
3,1.0,1.0,2.0,1.0,3.0,3.0,3.0,2.0,1.0,5.0,...,3.0,1.0,1.0,3.0,1.0,2.0,1.0,1.0,3.0,3.0
4,2.0,1.0,3.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,...,2.0,2.0,3.0,2.0,2.0,2.0,3.0,2.0,1.0,2.0


In [318]:
categorical_answers = responses.T
categorical_answers = categorical_answers[140:150].reset_index()
categorical_answers.head() 

,original,0,1,2,3,4,5,6,7,8,...,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009
0,Age,20.0,19.0,20.0,22.0,20.0,20.0,20.0,19.0,18.0,...,16.0,18.0,22.0,20.0,22.0,20.0,27.0,18.0,25.0,21.0
1,Height,163.0,163.0,176.0,172.0,170.0,186.0,177.0,184.0,166.0,...,153.0,160.0,181.0,172.0,175.0,164.0,183.0,173.0,173.0,185.0
2,Weight,48.0,58.0,67.0,59.0,59.0,77.0,50.0,90.0,55.0,...,62.0,63.0,72.0,63.0,60.0,57.0,80.0,75.0,58.0,72.0
3,How many siblings do you have?,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,2.0,1.0,1.0,1.0,1.0,5.0,0.0,1.0,1.0
4,Gender,female,female,female,female,female,male,female,male,female,...,female,female,male,female,male,female,male,female,female,male


Lets create a new dataframe so that we can visualize it with Tableau. 

In [370]:
# Creating dataframes for numeric question and categorical questions 
numeric_questions = columns['original'][0:140]
categorical_questions = columns['original'][140:150].reset_index(drop=True)

# We will create a temporary dataframe and then convert it into a final dataframe. 
# For every subsequent temporary dataframe, we will concatenate it to the final dataframe. 
# The final dataframe will also include new columns such as 'Count of Individuals', 'Gender', 'Age (bins)'. 

# For the temp dataframe, it will represent each column of numeric_answers. 
# Note: Each column represents a particpant and his/her answers. 

# Loop through each column in numeric_answers 
for i in range(numeric_answers.shape[1]):
    
    # create temp dataframe and insert numeric_questions as the initial column
    temp_df = pd.DataFrame(numeric_questions)
    
    # Rename the column to 'Questions (1-5)'
    temp_df.columns=['Questions (1-5)']
    
    # Create column called 'Answers' with all the answer for this particpant 
    temp_df['Answers'] = numeric_answers[i]
    
    # Categorize each answer to the corresponding questionnarie group. 
    questionnarie = []
    for row in temp_df['Questions (1-5)']:
        for preference, match in preference_dict.items():
            if row in match:
                questionnarie.append(preference)
                
    # Create Questionnarie Column
    temp_df['Questionnarie'] = questionnarie
    
    # Create categorical columns with the respective answers  
    for index, question in enumerate(categorical_questions):
        temp_df[question] = [categorical_answers[i][index]] * len(numeric_questions)
    
    # inital temp df will serve as the final dataframe base
    if i == 0:
        df = temp_df
    
    # for every subsequent temp dataframe, we will concat it.  
    else:
        df = pd.concat([df, temp_df]).reset_index(drop=True)
    
    
    # Create columns to be used as filters in Tableau 
    # Add Count of Individuals Column
    count = [1] * df.shape[0]
    df['Count of Individuals'] = count 
    
    # Capitalize gender type values 
    df['Gender'] = np.where(df['Gender'] == 'female', 'Female', df['Gender'])
    df['Gender'] = np.where(df['Gender'] == 'male', 'Male', df['Gender'])
    
    # Create Age Bin 
    age_bin = []
    for age in df['Age']:
        if age in list(range(27, 31)):
            age_bin.append('27-30')
            
        elif age in list(range(23, 27)):
            age_bin.append('23-26')
            
        elif age in list(range(19, 23)):
            age_bin.append('19-22')
            
        elif age in list(range(15, 19)):
            age_bin.append('15-18')
        
        else:
            age_bin.append(np.nan)
    
    # Add Age (bin) column
    df['Age (bin)'] = age_bin

We have successfully cleaned data so we can visualize it into Tableau. We can use each column to filter the data when visualizing it with Tableau. 

In [372]:
# sanity check
df

,Questions (1-5),Answers,Questionnarie,Age,Height,Weight,How many siblings do you have?,Gender,I am,Highest education achieved,I am the only child,I spent most of my childhood in a,I lived most of my childhood in a,Count of Individuals,Age (bin)
0,I enjoy listening to music.,5.0,Music Preference,20.0,163.0,48.0,1.0,Female,right handed,college/bachelor degree,no,village,block of flats,1,19-22
1,I prefer.,3.0,Music Preference,20.0,163.0,48.0,1.0,Female,right handed,college/bachelor degree,no,village,block of flats,1,19-22
2,"Dance, Disco, Funk",2.0,Music Preference,20.0,163.0,48.0,1.0,Female,right handed,college/bachelor degree,no,village,block of flats,1,19-22
3,Folk music,1.0,Music Preference,20.0,163.0,48.0,1.0,Female,right handed,college/bachelor degree,no,village,block of flats,1,19-22
4,Country,2.0,Music Preference,20.0,163.0,48.0,1.0,Female,right handed,college/bachelor degree,no,village,block of flats,1,19-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141395,I prefer branded clothing to non branded.,2.0,Spending Habits,21.0,185.0,72.0,1.0,Male,right handed,secondary school,no,village,house/bungalow,1,19-22
141396,I spend a lot of money on partying and social...,3.0,Spending Habits,21.0,185.0,72.0,1.0,Male,right handed,secondary school,no,village,house/bungalow,1,19-22
141397,I spend a lot of money on my appearance.,1.0,Spending Habits,21.0,185.0,72.0,1.0,Male,right handed,secondary school,no,village,house/bungalow,1,19-22
141398,I spend a lot of money on gadgets.,1,Spending Habits,21.0,185.0,72.0,1.0,Male,right handed,secondary school,no,village,house/bungalow,1,19-22


In [375]:
# Export dataframe to CSV
df.to_csv('survey.csv', index=False)